# Credit Card Marketing Campaign Simulation
This notebook simulates a targeted campaign using customer segmentation, A/B testing, and ROI analysis.

In [2]:
pip install ucimlrepo scikit-learn matplotlib pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load dataset
bank_marketing = fetch_ucirepo(id=222)
X = bank_marketing.data.features
y = bank_marketing.data.targets
df = pd.concat([X, y], axis=1)

In [5]:
# Clustering
cluster_features = df[['age', 'balance', 'duration', 'campaign']].copy()
scaler = StandardScaler()
scaled_features = scaler.fit_transform(cluster_features)
kmeans = KMeans(n_clusters=4, random_state=42)
df['segment'] = kmeans.fit_predict(scaled_features)

C:\Users\amand\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\amand\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\amand\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\amand\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\amand\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

In [ ]:
# A/B Test Simulation
np.random.seed(42)
df['group'] = np.random.choice(['A', 'B'], size=len(df), p=[0.5, 0.5])
df['converted'] = df.apply(lambda row: np.random.rand() < 0.02 if row['group'] == 'A' else np.random.rand() < 0.05, axis=1)
df['spend'] = df['converted'].apply(lambda x: np.random.normal(1000, 300) if x else 0)

In [ ]:
# Summary statistics
summary = df.groupby(['group'])[['converted', 'spend']].agg(
    conversion_rate=('converted', 'mean'),
    avg_spend=('spend', lambda x: x[x > 0].mean()),
    total_conversions=('converted', 'sum'),
    total_revenue=('spend', 'sum')
).reset_index()
summary

In [ ]:
# ROI and lift analysis
cost_per_person = 2
group_a = summary[summary['group'] == 'A']
group_b = summary[summary['group'] == 'B']

conv_rate_a = group_a['conversion_rate'].values[0]
conv_rate_b = group_b['conversion_rate'].values[0]
revenue_a = group_a['total_revenue'].values[0]
revenue_b = group_b['total_revenue'].values[0]
avg_spend_a = group_a['avg_spend'].values[0]

n_a = df[df['group'] == 'A'].shape[0]
n_b = df[df['group'] == 'B'].shape[0]

conversion_lift = conv_rate_b - conv_rate_a
revenue_lift = revenue_b - revenue_a
campaign_cost = n_b * cost_per_person
expected_revenue_b = conv_rate_a * n_b * avg_spend_a
incremental_revenue = revenue_b - expected_revenue_b
roi = incremental_revenue / campaign_cost

print("--- Campaign Performance Summary ---")
print(f"Conversion Rate A (Control): {conv_rate_a:.2%}")
print(f"Conversion Rate B (Treatment): {conv_rate_b:.2%}")
print(f"Conversion Lift: {conversion_lift:.2%}")
print(f"Revenue Lift: ${revenue_lift:,.2f}")
print(f"Campaign Cost: ${campaign_cost:,.2f}")
print(f"Incremental Revenue: ${incremental_revenue:,.2f}")
print(f"ROI: {roi:.2f}")